In [1]:
import openai
from transformers import AutoModelForCausalLM, AutoTokenizer
import argparse
import torch

In [2]:




initial_prompt = """
Context: You are a participant in a memory experiment. First, you will be given a list of words to memorized. After that, you will go through and complete a short list of math problems. Later, you will complete a recognition test with test list. Some of the words in the test lists may have appeared in the study list I initially asked you to memorize. Your task is as follows: if you remember seeing a word in the earlier phase of the experiment, label it as "old." If you believe it did not appear in the study list, label it as "new." For words labeled "old," if you have a vivid memory of the actual presentation, mark it with "K." If you are sure the word was presented in the study list but lack the feeling of recalling its actual occurrence, mark it with "R."

Instructions:

Study List of Words: Here is the list of words for you to remember. Take a moment to go through them carefully.

Study List: [mad, fear, hate, rage, temper, fury, ire, wrath, happy, fight, hatred, mean, calm, emotion, enrage, white, dark, cat, charred, night, funeral, color, grief, blue, death, ink, bottom, coal, brown, gray, butter, food, eat, sandwich, rye, jam, milk, flour, jelly, dough, crust, slice, wine, loaf, toast, table, sit, legs, seat, couch, desk, recliner, sofa, wood, cushion, swivel, stool, sitting, rocking, bench, hot, snow, warm, winter, ice, wet, frigid, chilly, heat, weather, freeze, air, shiver, Arctic, frost, nurse, sick, lawyer, medicine, health, hospital, dentist, physician, ill, patient, office, stethoscope, surgeon, clinic, cure, shoe, hand, toe, kick, sandals, soccer, yard, walk, ankle, arm, boot, inch, sock, smell, mouth, apple, vegetable, orange, kiwi, citrus, ripe, pear, banana, berry, cherry, basket, juice, salad, bowl, cocktail]

Math Problems: Solve these 5 simple math questions to ensure a brief delay before the test.

Math Questions:

5 + 3
9 - 2
4 * 2
8 / 4
7 - 5

Recognition Test: For each word in this list, please decide:
Old or New: Whether you think the word was presented in the study list of the study phase (if it was, mark it as "old"; if not, mark it as "new").
Remember or Know: If you judge the word as "old," further specify:
"R" (Remember): if you have a vivid memory of the actual presentation.
"K" (Know): if you are sure the word was presented but lack a vivid memory of it.

Recognition Test List: [mad, ire, enrage, white, color, gray, butter, milk, toast, table, recliner, bench, hot, frigid, frost, nurse, dentist, cure, shoe, yard, mouth, apple, pear, cocktail, anger, black, bread, chair, cold, doctor, foot, fruit, girl, high, king, man, boy, niece, sister, low, building, elevate, queen, throne, reign, woman, strong, old]

Example Response Format: For each word, provide a response in the format in order: 1, Old/New - R/K (if Old).

Your Task: Please rmemorize the study list, go through the math questions, and then proceed with the recognition test using the format provided.
"""

In [3]:
prompt = """

Goal: Memorizing the study list, where all sentences are in the format of "The [person] is in the [location]", answer all 76 questions in the questions list (questions are in the form of "Is the [person] in the [location]?") based on the study list. If the person-location pair is found in the study list, answer "yes"; otherwise, answer "no." Please also indicate the order.

Study list is [  
The actor is in the airport,
The chef is in the attic,
The coach is in the bank,
The cowboy is in the barn,
The dancer is in the castle,
The detective is in the church,
The doctor is in the clinic,
The engineer is in the factory,
The farmer is in the garage,
The gardener is in the hallway,
The hippie is in the hotel,
The inventor is in the kitchen,
The judge is in the laboratory,
The maid is in the library,
The musician is in the museum,
The pilot is in the nightclub,
The pirate is in the office,
The queen is in the park,
The scientist is in the school,
The sheriff is in the prison,
The pirate is in the park,
The queen is in the office,
The scientist is in the prison,
The sheriff is in the school,
The teacher is in the temple,
The tourist is in the theater,
The soldier is in the stadium,
The spy is in the studio,
The teacher is in the theater,
The tourist is in the temple,
The soldier is in the studio,
The spy is in the stadium,
]. 
    
Question list is [
Is the actor in the airport?
Is the chef in the attic?
Is the coach in the bank?
Is the cowboy in the barn?
Is the dancer in the castle?
Is the detective in the church?
Is the doctor in the clinic?
Is the engineer in the factory?
Is the farmer in the garage?
Is the gardener in the hallway?
Is the hippie in the hotel?
Is the inventor in the kitchen?
Is the judge in the laboratory?
Is the maid in the library?
Is the musician in the museum?
Is the pilot in the nightclub?
Is the pirate in the office?
Is the queen in the park?
Is the scientist in the school?
Is the sheriff in the prison?
Is the pirate in the park?
Is the queen in the office?
Is the scientist in the prison?
Is the sheriff in the school?
Is the teacher in the temple?
Is the tourist in the theater?
Is the soldier in the stadium?
Is the spy in the studio?
Is the teacher in the theater?
Is the tourist in the temple?
Is the soldier in the studio?
Is the spy in the stadium?
Is the actor in the nightclub?
Is the chef in the museum?
Is the coach in the library?
Is the cowboy in the laboratory?
Is the dancer in the kitchen?
Is the detective in the hotel?
Is the doctor in the hallway?
Is the engineer in the garage?
Is the farmer in the factory?
Is the gardener in the clinic?
Is the hippie in the church?
Is the inventor in the castle?
Is the judge in the barn?
Is the maid in the bank?
Is the musician in the attic?
Is the pilot in the airport?
Is the pirate in the school?
Is the queen in the prison?
Is the scientist in the office?
Is the sheriff in the park?
Is the pirate in the prison?
Is the queen in the school?
Is the scientist in the park?
Is the sheriff in the office?
Is the teacher in the stadium?
Is the tourist in the studio?
Is the soldier in the temple?
Is the spy in the theater?
Is the teacher in the studio?
Is the tourist in the stadium?
Is the soldier in the theater?
Is the spy in the temple?
].

The answers will be provided in CSV format.

"""

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)


messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt},
]


inputs = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt",
)

model = AutoModelForCausalLM.from_pretrained(model_id, 
                                             torch_dtype=torch.bfloat16, 
                                             device_map="auto")


inputs.to(model.device)
outputs = model.generate(**inputs, 
                         max_new_tokens=5000)
                         #temperature=0.6,
                         #top_p=0.9)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

2025-04-21 21:19:39.009400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-21 21:19:39.022864: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-21 21:19:39.027198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-21 21:19:39.037887: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-21 21:19:39.764522: W tensorflow/compiler/tf2

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


You are a helpful assistant.



Goal: Memorizing the study list, where all sentences are in the format of "The [person] is in the [location]", answer all 76 questions in the questions list (questions are in the form of "Is the [person] in the [location]?") based on the study list. If the person-location pair is found in the study list, answer "yes"; otherwise, answer "no." Please also indicate the order.

Study list is [  
The actor is in the airport,
The chef is in the attic,
The coach is in the bank,
The cowboy is in the barn,
The dancer is in the castle,
The detective is in the church,
The doctor is in the clinic,
The engineer is in the factory,
The farmer is in the garage,
The gardener is in the hallway,
The hippie is in the hotel,
The inventor is in the kitchen,
The judge is in the laboratory,
The maid is in the library,
The musician is in the museum,
The pilot is in the nightclub,
The pirate is in the office,
The queen is in the park,
The scientist is in the school,
The sheriff i